In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[["charges"]]

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
indep=sc.fit_transform(independent)


In [11]:
indep

array([[-1.43876426, -0.45332   , -0.90861367, -1.0105187 ,  1.97058663],
       [-1.50996545,  0.5096211 , -0.07876719,  0.98959079, -0.5074631 ],
       [-0.79795355,  0.38330685,  1.58092576,  0.98959079, -0.5074631 ],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -1.0105187 , -0.5074631 ],
       [-1.29636188, -0.79781341, -0.90861367, -1.0105187 , -0.5074631 ],
       [ 1.55168573, -0.26138796, -0.90861367, -1.0105187 ,  1.97058663]])

In [16]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel': ['rbf','linear', 'poly'],
    'C': [1, 10, 100],
    'gamma': ['scale', 'auto']}
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose= 3,n_jobs=-1)
grid.fit(indep, dependent)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'linear', 'poly']},
             verbose=3)

In [17]:
re=grid.cv_results_
grid_predictions = grid.predict(indep)
from sklearn.metrics import r2_score
r_score=r2_score(dependent,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)


The R_score value for best parameter {'C': 100, 'gamma': 'auto', 'kernel': 'poly'}: 0.6791935915962988


In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.328422,0.067612,0.112775,0.008139,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",-0.108985,-0.106259,-0.075060,-0.099683,-0.100792,-0.098156,0.012047,17
1,0.145524,0.004450,0.028334,0.005880,1,scale,linear,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",-0.025629,-0.011630,0.007866,-0.014337,-0.027381,-0.014222,0.012632,11
2,0.150686,0.005765,0.027686,0.004262,1,scale,poly,"{'C': 1, 'gamma': 'scale', 'kernel': 'poly'}",-0.099348,-0.096181,-0.066486,-0.091099,-0.092252,-0.089073,0.011666,16
3,0.196261,0.006557,0.101870,0.002003,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",-0.108974,-0.106280,-0.075075,-0.099676,-0.100778,-0.098157,0.012041,18
4,0.157097,0.008903,0.022511,0.005596,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",-0.025629,-0.011630,0.007866,-0.014337,-0.027381,-0.014222,0.012632,11
5,0.150599,0.004025,0.027027,0.003809,1,auto,poly,"{'C': 1, 'gamma': 'auto', 'kernel': 'poly'}",-0.099561,-0.095837,-0.066222,-0.091226,-0.092426,-0.089054,0.011780,15
6,0.189712,0.004117,0.100792,0.005767,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",-0.050517,-0.045096,-0.016601,-0.041415,-0.048600,-0.040446,0.012321,13
7,0.154866,0.010646,0.025854,0.004100,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.487873,0.472341,0.485102,0.475835,0.447692,0.473769,0.014239,5
8,0.147160,0.003816,0.028109,0.005415,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.039041,0.040111,0.063362,0.042506,0.036594,0.044323,0.009707,9
9,0.183677,0.005558,0.096368,0.004605,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",-0.050402,-0.045276,-0.016737,-0.041345,-0.048491,-0.040450,0.012248,14


In [20]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 25
BMI: 25
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [21]:
Insurance_charges=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Insurance Charges={}".format(Insurance_charges))


Insurance Charges=[14612283.96694745]
